In [9]:
from astropy.table import Table
import pandas as pd
import time as t
import numpy as np

In [2]:
base = '/global/cfs/cdirs/desi/public/dr1/spectro/redux/iron/zcatalog/v1'

In [3]:
ini = t.time()
cat = Table.read(f'{base}/zall-pix-iron.fits')
end = t.time()
len(cat)

28425963

In [5]:
cat[:5]

TARGETID,SURVEY,PROGRAM,HEALPIX,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,FA_TARGET,FA_TYPE,OBJTYPE,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV1_SCND_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV2_SCND_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,PLATE_RA,PLATE_DEC,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_PSF_TO_FIBER_SPECFLUX,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MIN_MJD,MAX_MJD,MEAN_MJD,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,MAIN_NSPEC,MAIN_PRIMARY,SV_NSPEC,SV_PRIMARY,ZCAT_NSPEC,ZCAT_PRIMARY,DESINAME
,,,,,,,,,,,,,,,,deg,deg,mas / yr,mas / yr,yr,,,,,,,,,,,mag,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,,,arcsec,,,,,mag,mag,mag,mas,,,,,,,,,,,,,,,,,,,,,deg,deg,,s,,,mm,mm,mm,mm,,deg,arcsec,deg,arcsec,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,bytes7,bytes6,int32,int32,float64,float64,int64,float64,float64[10],int64,bytes6,bytes20,int64,float64,int32,float64,float64,float32,float32,float32,int64,uint8,bytes3,float64,int32,int16,bytes8,int32,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,bytes2,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int16,float32,int16,int16,float32,float32,float32,float32,float32,float64,float32,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,bool,int16,bool,int16,bool,bytes22
39628473198710603,cmx,other,2152,2152,0.8042057637744875,9.589004434794145e-06,0,10003.721982955933,117.8692939130018 .. 5.6065445634658255,7928,GALAXY,--,10,6319.3509159088135,0,23.764862479118218,29.832378962505196,0.0,0.0,2020.9597,3072,1,TGT,0.20970260241473437,3,9010,0237p297,494512,3915,EXP,0.05433502,1.3540096,2.9762945,10.3957405,27.52511,48.7791,531.4606,123.64128,19.293917,2.6645064,0.6410277,0.4344953,0.95507884,3.3359442,0.43980038,0.9776775,3.4928923,0,1.0,1.1636901,0.18920432,-0.24198028,0,--,0.0,0.0,0.0,0.0,S,3200,1,3072,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23.764862479118218,29.832378962505196,4,3600.0,1,1,-0.00675,0.0091515025,-0.0055,0.014124447,0.71950334,23.76489350625343,0.08943564,29.832358617984326,0.1715876,59200.06640136,59200.12381137,59200.095110124996,388.2487,0.28193328,75.31641,283.94504,1451.2411,530.06067,7.442073,2.1379867,34921.81,78.42465,6280.0347,0.1178768,7149.4326,36527.004,22.981281,113.36229,5.09169e-05,260.3999,9.6547765e-06,0.0,11533.531,6.420885e-05,52.333626,120.35188,35310.06

In [4]:
(end-ini)/60

1.979544468720754

In [7]:
for c in cat.columns:
    print(c)

TARGETID
SURVEY
PROGRAM
HEALPIX
SPGRPVAL
Z
ZERR
ZWARN
CHI2
COEFF
NPIXELS
SPECTYPE
SUBTYPE
NCOEFF
DELTACHI2
COADD_FIBERSTATUS
TARGET_RA
TARGET_DEC
PMRA
PMDEC
REF_EPOCH
FA_TARGET
FA_TYPE
OBJTYPE
SUBPRIORITY
OBSCONDITIONS
RELEASE
BRICKNAME
BRICKID
BRICK_OBJID
MORPHTYPE
EBV
FLUX_G
FLUX_R
FLUX_Z
FLUX_W1
FLUX_W2
FLUX_IVAR_G
FLUX_IVAR_R
FLUX_IVAR_Z
FLUX_IVAR_W1
FLUX_IVAR_W2
FIBERFLUX_G
FIBERFLUX_R
FIBERFLUX_Z
FIBERTOTFLUX_G
FIBERTOTFLUX_R
FIBERTOTFLUX_Z
MASKBITS
SERSIC
SHAPE_R
SHAPE_E1
SHAPE_E2
REF_ID
REF_CAT
GAIA_PHOT_G_MEAN_MAG
GAIA_PHOT_BP_MEAN_MAG
GAIA_PHOT_RP_MEAN_MAG
PARALLAX
PHOTSYS
PRIORITY_INIT
NUMOBS_INIT
CMX_TARGET
DESI_TARGET
BGS_TARGET
MWS_TARGET
SCND_TARGET
SV1_DESI_TARGET
SV1_BGS_TARGET
SV1_MWS_TARGET
SV1_SCND_TARGET
SV2_DESI_TARGET
SV2_BGS_TARGET
SV2_MWS_TARGET
SV2_SCND_TARGET
SV3_DESI_TARGET
SV3_BGS_TARGET
SV3_MWS_TARGET
SV3_SCND_TARGET
PLATE_RA
PLATE_DEC
COADD_NUMEXP
COADD_EXPTIME
COADD_NUMNIGHT
COADD_NUMTILE
MEAN_DELTA_X
RMS_DELTA_X
MEAN_DELTA_Y
RMS_DELTA_Y
MEAN_PSF_TO_FIBE

In [10]:
np.unique(cat['OBJTYPE'])

""
BAD
NON
SKY
TGT


In [13]:
cols = ['name','ra','dec','z_lens','z_source','zl_phot','zs_phot','Rein','lens_mag_i','source_mag_i','type','discovery','grade','Reference']
hsc = pd.read_csv('/pscratch/sd/v/vtorresg/desi-lenses/hsc_full_catalog.csv', header=None, names=cols)

In [18]:
row_h = hsc.iloc[0]

In [19]:
row_z = cat[0]

Necesito hacer crossmatching de los dos catalogos

- hsc tiene ra,dec
- en el paper definen un radio de 6''
- puedo tomar las coordenadas de cada lente para tomarlas como centro de un circulo de radio 6''
- luego puedo filtrar el catalogo de z

------------
- puedo calcular la distancia entre el lente y los objetos de desi
- calcular la distancia ra1,dec1 ra2,dec2 y ver si tienen una separacion angular de <= 6''

In [21]:
from astropy.coordinates import SkyCoord
import astropy.units as u

In [24]:
p_h = SkyCoord(ra=row_h['ra']*u.deg, dec=row_h['dec']*u.deg)
p_r = SkyCoord(ra=row_z['TARGET_RA']*u.deg, dec=row_z['TARGET_DEC']*u.deg)

$$\ cos \theta = \sin \delta_1 \sin \delta_2 + \cos \delta_1 \cos \delta_2 \cos \left( \alpha_1 - \alpha_2 \right)$$

In [25]:
d = p_h.separation(p_r)
d

<Angle 35.44450169 deg>

skycoord tiene un metodo search_around_sky

In [30]:
#la tabla ya esta en grados?
d_coord = SkyCoord(ra=cat['TARGET_RA'], dec=cat['TARGET_DEC'])

In [35]:
h_coord = SkyCoord(ra=hsc['ra'].to_numpy()*u.deg, dec=hsc['dec'].to_numpy()*u.deg)

In [37]:
d_coord.shape

(28425963,)

In [41]:
row = h_coord[0]
row

<SkyCoord (ICRS): (ra, dec) in deg
    (0.03524, 2.44944)>

In [42]:
idx_d, idx_h, sep2d, _ = d_coord.search_around_sky(h_coord, seplimit=6*u.arcsec)

In [43]:
idx_d #index de los objetos de desi

array([   2,    3,    5, ..., 3955, 3956, 3956])

In [44]:
idx_h #index de objetos de hsc

array([17079632, 17079353, 18560974, ..., 18061521,  7765506, 18566001])

In [45]:
sep2d #sep entre los pares

<Angle [1.06917721e-05, 1.40911073e-05, 1.69098141e-05, ...,
        4.97801121e-06, 2.04684749e-07, 2.04684749e-07] deg>

In [46]:
_ #esta seria la dis

<Quantity [1.86606627e-07, 2.45936218e-07, 2.95131932e-07, ...,
           8.68826857e-08, 3.57242280e-09, 3.57242280e-09]>